In [ ]:
#------Importing Packages-----#
import warnings
warnings.filterwarnings('ignore')
import scanpy as sc
import anndata as ad
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy
import os
from scipy.io import mmread
from scipy.sparse import csr_matrix
import seaborn as sns
#Spatial data imports
import spatialdata_io
import spatialdata as sd
import spatialdata_plot
from spatialdata import get_extent
from spatialdata import bounding_box_query

In [ ]:
#--------Generating Spatial Object: Donor 1-------#
#Using grayscale converted image
sdata = spatialdata_io.visium_hd(path = "/projects/b1217/Edward/Spatial/VisiumHD/Human/20240612EL_HuScalp_V2/outs",
                                 bin_size = [8],
                                 bins_as_squares=True,
                                 fullres_image_file="/projects/b1217/Edward/Spatial/VisiumHD/Human/20240612EL_HuScalp_V2/outs/spatial/20241020_huscalp_EL_20x_tiled_HE_BWyellow.tif",)
#------Writing to zarr-----#
sdata.write("/projects/b1217/HHA/Spatial/Zarrs/EL_S2", overwrite = True)
sdata = sd.read_zarr("/projects/b1217/HHA/Spatial/Zarrs/EL_S2")
sdata

In [ ]:
#-------Plotting H&E Image: Full and Low Resolution------#
axes = plt.subplots(1, 2, figsize=(10, 5))[1].flatten()
sdata.pl.render_images('20240612EL2_full_image').pl.show(ax=axes[0], title="Full", coordinate_systems="global")
sdata.pl.render_images("20240612EL2_lowres_image").pl.show(ax=axes[1], title="Lowres", coordinate_systems="global")

In [ ]:
#-------Formatting Gene Names-------#
#creates ensembl and symbol ID columns and replaces index with ENSEMBL
sdata["square_008um"].var["ensembl_id"] = sdata["square_008um"].var["gene_ids"]
sdata["square_008um"].var['symbol'] = sdata["square_008um"].var_names
sdata["square_008um"].var.set_index('symbol', drop=True, inplace=True)
sdata["square_008um"].var_names_make_unique()

In [ ]:
#----------Loading in Mapped Matrix Data----------#
Matrix_spatial = sc.read("/projects/b1217/HHA/Bulb_Spatial/20250516_HHA_Spatial_TACCO_Matrix_Subset.h5ad")
Matrix_spatial

In [ ]:
#------Filtering Data for Spots in Matrix-----#
#Adding barcodes to metadata
sdata.tables["square_008um"].obs['barcode'] = sdata.tables["square_008um"].obs_names
print(sdata.tables["square_008um"].obs.head())
#Pulling anndata from object
adata = sdata.tables["square_008um"]
#Getting filter for cells in the upper HF/SG
mask = adata.obs["barcode"].isin(Matrix_spatial.obs_names).values
#Filtering AnnData
adata_sub = adata[mask, :].copy()
adata_sub

#-----Matching Spot Order-----#
Matrix_spatial = Matrix_spatial[adata_sub.obs_names,:]
print(np.sum(Matrix_spatial.obs_names == adata_sub.obs_names))

#------Adding metadata to subsetted anndata------#
adata_sub.obs = adata_sub.obs.join(Matrix_spatial.obs, how = "left", on = "barcode", lsuffix = "L_", rsuffix = "R_")

#-----Adding new AnnData to SData Object-----#
sdata.tables["square_008um"] = adata_sub
sdata.tables["square_008um"]

#------Filtering shapes to those in mask------#
#list of shapes found in the filtered spatial object
keeplist = sdata.tables["square_008um"].obs["location_id"].unique()
#All shapes
sh = sdata.shapes['20240612EL2_square_008um']
#Filtering for shapes in anndata
mask = sh.index.isin(keeplist)
sh_sub = sh[mask].copy()
sdata.shapes['20240612EL2_square_008um'] = sh_sub

In [ ]:
#--------Adding in Lower versus Upper AUCell Scores-------#
DA_AUCell = pd.read_csv("/projects/b1217/HHA/Bulb_Spatial/HHA_Spatial_Lower_Upper_DA_AUCell_6_11_25.csv")
DA_AUCell.index = DA_AUCell["barcode"]
#Adding to sdata
sdata.tables["square_008um"].obs["Lower_DA"] = DA_AUCell["Lower_DA"]
sdata.tables["square_008um"].obs["Upper_DA"] = DA_AUCell["Upper_DA"]

In [ ]:
#------Plotting TACCO Predicted CellTypes-----#
#Setting Palette and Plotting Order
MatrixPal = ["#89C75F", "#3BBCA8", "#208A42", "#0C727C", "#9ECAE1", "#4292C6", "#08306B",
        "#E6C2DC", "#C06CAB", "#89288F", "#D8A767", "#F47D2B", "#F37B7D", "#7E1416", "#D24B27"]
matrix_groups = ["Lower COL17", "Upper COL17", "LPC", "Medulla", "Early_Cortex", "Middle_Cortex", "Late_Cortex",
             "Early_Cuticle", "Middle_Cuticle", "Late_Cuticle",
             "Early_IRS_I", "Early_IRS_II", "IRS_Henle", "IRS_Huxley", "IRS_Cuticle"]

#------Plotting on Full Image-------#
fig,ax=plt.subplots(figsize=(40, 40))
image = '20240612EL2_full_image'
shapes = '20240612EL2_square_008um'
#Plotting Clusters on Full Image
sdata.pl.render_images(image).pl.render_shapes(
  shapes, 
  color="MatrixAnnotationFine",
  palette = MatrixPal,
    groups = matrix_groups,
  method="matplotlib").pl.show(coordinate_systems="global", title=f"bin_size=8µm", ax=ax)
#Saving as png
fig.savefig("/projects/b1217/HHA/Bulb_Seurat_Plots/Spatial/TACCO_Mapping_Spatial_Grayscale.png",
            dpi = 600, pad_inches = 0.5, bbox_inches = "tight")

In [ ]:
#------Plotting AUCell Scores: Lower_DA------#
fig,ax=plt.subplots(figsize=(40, 40))
image = '20240612EL2_full_image'
shapes = '20240612EL2_square_008um'
#Plotting Clusters on Full Image
sdata.pl.render_images(image).pl.render_shapes(
  shapes, 
  color="Lower_DA",
  cmap = "magma",
  method="matplotlib").pl.show(coordinate_systems="global", title=f"bin_size=8µm", ax=ax)
#Saving as png
fig.savefig("/projects/b1217/HHA/Bulb_Seurat_Plots/Spatial/NMF_Plots/Matrix_Spatial_AUCell_Lower_DA.png",
            dpi = 600, pad_inches = 0.5, bbox_inches = "tight")

In [ ]:
#------Plotting AUCell Scores: Upper_DA------#
fig,ax=plt.subplots(figsize=(40, 40))
image = '20240612EL2_full_image'
shapes = '20240612EL2_square_008um'
#Plotting Clusters on Full Image
sdata.pl.render_images(image).pl.render_shapes(
  shapes, 
  color="Upper_DA",
  cmap = "magma",
  method="matplotlib").pl.show(coordinate_systems="global", title=f"bin_size=8µm", ax=ax)